# Context
Goal of project is to classify resumes (not to grade resumes). Therefore, goal of preprocessing is to ensure that the text are properly normalized such that they can be properly compared. Priority should therefore be given to keywords etc. due to the specialized nature of each class.

# Data Exploration

In [1]:
import numpy
import numpy as np
import pandas as pd
import re
import nltk
import os
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.metrics.distance import edit_distance as levenshteinDistance

from typing_extensions import Literal, Iterable

C:\Users\JS\AppData\Local\Temp\ipykernel_61204\2766822271.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
VAR = {
    'data_path': os.path.join('UpdatedResumeDataSet_T1_7.csv'),
    'batch_size': 32,
}

In [4]:
res_data_raw = pd.read_csv(VAR['data_path'])

In [5]:
res_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9595 entries, 0 to 9594
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  9595 non-null   object
 1   Resume    9595 non-null   object
dtypes: object(2)
memory usage: 150.1+ KB


In [6]:
res_data_raw.head(5)

,Category,Resume
0,Data Science,qwtnrvduof Education Details \nMay 2013 to May...
1,Data Science,"qwtnrvduof Areas of Interest Deep Learning, Co..."
2,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
3,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
4,Data Science,"SKILLS C Basics, IOT, Python, MATLAB, Data Sci..."


### Cleaning

In [7]:
res_data_raw['Resume'].duplicated(keep='first').sum() #Check for duplicates

9407

In [8]:
res_data = res_data_raw.drop_duplicates(subset=['Resume'])

In [9]:
res_data['Category'].value_counts()

Category
Java Developer               14
Data Science                 12
HR                           12
Database                     11
Advocate                     10
DotNet Developer              8
Hadoop                        8
DevOps Engineer               8
Business Analyst              8
Testing                       8
Civil Engineer                7
SAP Developer                 7
Health and fitness            7
Python Developer              7
Arts                          7
Automation Testing            7
Electrical Engineering        6
Sales                         6
Network Security Engineer     6
ETL Developer                 6
Mechanical Engineer           5
Web Designing                 5
Blockchain                    5
Operations Manager            4
PMO                           4
Name: count, dtype: int64

### Explore Resume Text

In [10]:
sample_res = res_data['Resume'][0]
print(sample_res)

qwtnrvduof Education Details 
May 2013 to May 2017 BbNTGBqLmkKE   UIT-RGPV
Data Scientist 

Data Scientist - Matelabs
Skill Details 
Python- Exprience - Less than 1 year months
Statsmodels- Exprience - 12 months
AWS- Exprience - Less than 1 year months
Machine learning- Exprience - Less than 1 year months
Sklearn- Exprience - Less than 1 year months
Scipy- Exprience - Less than 1 year months
Keras- Exprience - Less than 1 year monthsCompany Details 
company - Matelabs
description - ML Platform for business professionals, dummies and enthusiastsckeKJOFvWQ
60/A Koramangala 5th block,
Achievements/Tasks behind sukh sagar, Bengaluru,
India                               Developed and deployed auto preprocessing steps of machine learning mainly missing value
treatment, outlier detection, encoding, scaling, feature selection and dimensionality reductionqunsOBcUdT
Deployed automated classification and regression modelRYNOolXhuV
linkedinSAJhwmUxoOcom/in/aditya-rathore-
b4600b146                

Initial Review:
Some words are seemingly gibberish and consists of a sequence of random characters

These words should be removed. However, care must be taken to ensure that other important text such as links are not classified as gibberish

List of issues:
Broken links (Solved)
Long whitespaces (Solved)
Combined words without clear separators https://github.com/grantjenks/python-wordsegment

In [11]:
sample_res

'qwtnrvduof Education Details \nMay 2013 to May 2017 BbNTGBqLmkKE   UIT-RGPV\nData Scientist \n\nData Scientist - Matelabs\nSkill Details \nPython- Exprience - Less than 1 year months\nStatsmodels- Exprience - 12 months\nAWS- Exprience - Less than 1 year months\nMachine learning- Exprience - Less than 1 year months\nSklearn- Exprience - Less than 1 year months\nScipy- Exprience - Less than 1 year months\nKeras- Exprience - Less than 1 year monthsCompany Details \ncompany - Matelabs\ndescription - ML Platform for business professionals, dummies and enthusiastsckeKJOFvWQ\n60/A Koramangala 5th block,\nAchievements/Tasks behind sukh sagar, Bengaluru,\nIndia                               Developed and deployed auto preprocessing steps of machine learning mainly missing value\ntreatment, outlier detection, encoding, scaling, feature selection and dimensionality reductionqunsOBcUdT\nDeployed automated classification and regression modelRYNOolXhuV\nlinkedinSAJhwmUxoOcom/in/aditya-rathore-\nb46

In [12]:
def clean_links(potentialLinks: list):
    
    '''
    Assumption: Potential link will always have at the minimum a .com
    
    Checks validity of link and returns cleaned link string
    '''
    
    assert isinstance(potentialLinks, list)
    
    http_exist = False
    www_exist = False
    com_exist = False
    
    if len(potentialLinks) < 1:
        return []
    
    ret_list = []
    
    for link in potentialLinks:
        
        http_match = re.search(r'(https?)(:)?(\/){0,2}', link)
        www_match = re.search(r'(www)(\.)?', link)
        com_match = re.search(r'(\.)?(com)', link)
        # print('flagged', link)
        
        #http
        if http_match != None:
            http_exist = True
        
        #www
        if www_match != None:
            www_exist = True
        
        #com
        if com_match != None:
            com_exist = True
            
        if (com_exist) or (com_exist and www_exist) or (com_exist and www_exist and http_exist):
            link = re.sub(r'(https?)(:)?(\/){0,2}', 'https://', link)
            link = re.sub(r'(www)(\.)?', 'www.', link)
            link = re.sub(r'(\.)?(com)', '.com', link)
            
            ret_list.append(link)
        else:
            #Not valid link
            ret_list.append(False)
            
    return ret_list

In [13]:
def clean_raw_text(text: str):
    
    # Clean links section
    potential_links = re.findall(
        r'(?:(?:https?:?\/\/{1,2})?w{1,3}\.?)?[a-zA-z0-9]{1,2048}\.?[a-zA-Z0-9]{1,6}\/\b[/\-a-zA-Z0-9]*\w', text
    ) 
    '''
    / will flag a sequence of characters as potential links
    
    Optional criteions: 
    http(s)
    //
    www & .
    . & com
    '''
    
    finalized_links = clean_links(potential_links)

    for potential_link, finalized_link in zip(potential_links, finalized_links):
        if finalized_link == False:
            continue
        else:
#             print('real_links', finalized_link)
            text = re.sub(potential_link, ' ', text) #Remove link
    
    #Clean non-characters
    text = re.sub(r'[^a-zA-Z0-9]', r' ', text)
    
    #Normalize text
    text = text.lower()

    #Clean whitespace section
    text = re.sub(r'[ ]{1,}', r' ', text)
    
    return text

clean_raw_text(sample_res)

'qwtnrvduof education details may 2013 to may 2017 bbntgbqlmkke uit rgpv data scientist data scientist matelabs skill details python exprience less than 1 year months statsmodels exprience 12 months aws exprience less than 1 year months machine learning exprience less than 1 year months sklearn exprience less than 1 year months scipy exprience less than 1 year months keras exprience less than 1 year monthscompany details company matelabs description ml platform for business professionals dummies and enthusiastsckekjofvwq 60 a koramangala 5th block achievements tasks behind sukh sagar bengaluru india developed and deployed auto preprocessing steps of machine learning mainly missing value treatment outlier detection encoding scaling feature selection and dimensionality reductionqunsobcudt deployed automated classification and regression modelrynoolxhuv b4600b146 reasearch and deployed the time series forecasting model arima sarimax holt winter and prophetiqmadshiyn worked on meta feature

In [14]:
def check_numpy(text):
    
    if isinstance(text, list):
        text = np.array(text)
        return text
    elif isinstance(text, np.ndarray):
        return text
    else:
        raise TypeError('Not a list or numpy array')

In [15]:
def in_english_corpus(text: list | np.ndarray, behaviour: Literal['inside', 'outside'] = 'inside'):
    
    text = check_numpy(text)

    english_dictionary = nltk.corpus.words.raw().split('\n')

    english_dictionary = [word.lower() for word in english_dictionary] # normalize to lowercase
    
    word_in_dict_bool = np.isin(text, english_dictionary)
    
    if behaviour == 'inside':
        return text[word_in_dict_bool]
    elif behaviour == 'outside':
        word_not_in_dict_bool = np.invert(word_in_dict_bool)
        return text[word_not_in_dict_bool]
    else:
        return None

In [16]:
def clean_structured_text(text: list | np.ndarray, customer_dictionary: list = nltk.corpus.words.raw().split('\n')):
    
    #TODO There may be no point to cleaning mistyped random words > Intefere with keywords > Model may have to simply learn the noise

    text = check_numpy(text)
    
    customer_dictionary = [word.lower() for word in customer_dictionary] # normalize to lowercase
    
    word_in_dict_bool = np.isin(text, customer_dictionary)
    
    word_not_in_dict_bool = np.invert(word_in_dict_bool)
    
    
    
    words_in_dict = text[word_not_in_dict_bool]
    
    print(words_in_dict)

# clean_structured_text(sample_lemmas)

In [17]:
def wordnet_tag_format(tag: str):
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('V'):
        return 'v'
    if tag.startswith('A'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    
    return 'n' #Ensure lemmatize function can run

In [18]:
def extract_lemmas(tagged_tokens: list[tuple], lemmatizer=nltk.stem.WordNetLemmatizer()):
    lemmas = [lemmatizer.lemmatize(token[0], wordnet_tag_format(token[1])) for token in tagged_tokens]
    
    return lemmas

--- Start Analysis ---
### Goal:

With reference to a list of possible english words, we aim to separate keywords important to job scopes and misspelled/invalid words. With the list of misspelled/invalids words, we can find the closest possible related word using Levenshtein Distance.

### Theory:

Since most misspelled words with random number of combinations occuringly more than once has a very small probability it is more likely that we will see keywords occur more frequently compared to misspelled words and characters of random sequence.

In [19]:
def extract_common_words_from_raw_data_ood(resumes_df: pd.DataFrame, column: str):
    
    resumes = resumes_df[column].to_numpy()
    resumes = check_numpy(resumes)

    lemmatizer = WordNetLemmatizer()
    
    counter = Counter()
    
    for index, resume in enumerate(resumes):
        normalized_resume = extract_lemmas(
            nltk.pos_tag(
                nltk.tokenize.word_tokenize(
                    clean_raw_text(resume))), 
            lemmatizer)
        
        # out of dictionary
        ood = in_english_corpus(normalized_resume, 'outside')
        counter.update(ood)
        
        if index % 10 == 0:
            print(index)
        
    return counter

In [20]:
# count = extract_common_words_from_raw_data_ood(res_data, 'Resume')
# print(count)

#### Output Analysis

Clearly misspelled words like "exprience" occur most frequently, and sequence of seemingly random characters "bbntgbqlmkkeckekjofvwq" appeared 5 times.

Conversely, keywords like "mozilla" which may be important to Web Designers only appeared once. Other important keywords like "tensorflow" and "scikit" only appears 5 times, the same as "bbntgbqlmkkeckekjofvwq". I therefore hypothesise that words such as "bbntgbqlmkkeckekjofvwq" occurring is not based on chance due to the miniscule probability. There is thus no apparent clear threshold/boundary between misspelled/noise words and keywords.

--- End Analysis ---

In [21]:
def pipeline(filepath: str, feature_name: str):
    
    def total_normalize(text):
        text = clean_raw_text(text)
        text_tag = nltk.pos_tag(
            nltk.word_tokenize(text)
        )
        text_lemmas = extract_lemmas(text_tag)
        
        return ' '.join(text_lemmas)
    
    df = pd.read_csv(filepath)
    df = df.drop_duplicates(subset=[feature_name], keep='first')
    df[feature_name] = df[feature_name].apply(total_normalize)
    
    return df

In [22]:
processed_resumes = pipeline('UpdatedResumeDataSet_T1_7.csv', feature_name='Resume')
processed_resumes['Resume'][2]

'skill r python sap hana tableau sap hana sql sap hana pal m sql sap lumira c linear program data model advance analytics scm analytics retail analytics social medium analytics nlp education detail january 2017 to january 2018 pgdm business analytics great lake institute of management illinois institute of technology january 2013 bachelor of engineering electronics and communication bengaluru karnataka new horizon college of engineering bangalore visvesvaraya technological university data science consultant consultant deloitte usi skill detail linear program exprience 6 month retail exprience 6 month retail marketing exprience 6 month scm exprience 6 month sql exprience le than 1 year month deep learn exprience le than 1 year month machine learn exprience le than 1 year month python exprience le than 1 year month r exprience le than 1 year monthscompany detail company deloitte usi description the project involve analyse historic deal and come with insight to optimize future dealsbntgbq

In [23]:
print(processed_resumes['Category'].value_counts()) # Require at least 7 per class for trainTestSplit

Category
Java Developer               14
Data Science                 12
HR                           12
Database                     11
Advocate                     10
DotNet Developer              8
Hadoop                        8
DevOps Engineer               8
Business Analyst              8
Testing                       8
Civil Engineer                7
SAP Developer                 7
Health and fitness            7
Python Developer              7
Arts                          7
Automation Testing            7
Electrical Engineering        6
Sales                         6
Network Security Engineer     6
ETL Developer                 6
Mechanical Engineer           5
Web Designing                 5
Blockchain                    5
Operations Manager            4
PMO                           4
Name: count, dtype: int64


In [24]:
processed_resumes

,Category,Resume
0,Data Science,qwtnrvduof education detail may 2013 to may 20...
1,Data Science,qwtnrvduof area of interest deep learn control...
2,Data Science,skill r python sap hana tableau sap hana sql s...
3,Data Science,education detail mca ymcaust faridabad haryana...
4,Data Science,skill c basic iot python matlab data science m...
...,...,...
873,Testing,skill set o window xp 7 8 8bntgbqlmkk1 10 data...
874,Testing,good logical and analytical skill positive att...
878,Testing,personal skill quick learner eagerness to lear...
1540,DevOps Engineer,core skill project program management agile sc...


In [25]:
import random

minimum = 7
current_lowest = processed_resumes['Category'].value_counts().min()

#Check
count = processed_resumes['Category'].value_counts()
remaining = 7 - count[count<minimum]

while len(remaining != 0):
    count = processed_resumes['Category'].value_counts()
    remaining = 7 - count[count<minimum]
    
    for category in remaining.index:
        someInt = random.randint(0, current_lowest-1)
        value_to_append = processed_resumes[
            processed_resumes['Category']==category
        ]['Resume'].values[someInt]
        
        
        df_to_concat = pd.DataFrame({
            'Category': [category],
            'Resume': [value_to_append]
        })
    
        processed_resumes = pd.concat([processed_resumes, df_to_concat], axis=0)

# Classification Task: Initiating Data

In [26]:
def generate_class_labels(labels: Iterable):
    
    unique_labels = list(set(labels))
    
    unique_labels_map = {}
    
    for index, label in enumerate(unique_labels, start=0):
        unique_labels_map[label] = index
        
    return unique_labels_map

In [27]:
labels, features = processed_resumes['Category'], \
    processed_resumes['Resume'].to_numpy()

In [28]:
label_map = generate_class_labels(labels)

labels = labels.apply(lambda label_name: label_map[label_name])

labels = labels.to_numpy()

In [29]:
from transformers import BertTokenizer, BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from transformers import AdamW

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_resumes = [tokenizer.tokenize(feature) for feature in features]

assert len(tokenized_resumes) == processed_resumes.Resume.count()

In [33]:
max_len = 0
for tokenized_resume in tokenized_resumes:
    if len(tokenized_resume) > max_len:
        max_len = len(tokenized_resume)
    else:
        continue

In [34]:
max_len = 512 # TODO DELETE

In [35]:
input_ids = [
    tokenizer.convert_tokens_to_ids(tokenized_resume) for tokenized_resume in tokenized_resumes
]

assert len(input_ids[0]) == len(tokenized_resumes[0])

input_ids = pad_sequences(input_ids, maxlen=max_len, dtype='long', truncating='post', padding='post')

In [36]:
attention_masks_ = []
for resume_id in input_ids:
    resume_mask = [float(id > 0) for id in resume_id]
    attention_masks_.append(resume_mask)
    
assert len(attention_masks_[0]) == max_len

In [37]:
x_train, x_test_and_val, y_train, y_test_and_val, z_train, z_test_and_val = train_test_split(
    input_ids, labels, attention_masks_, test_size=0.3, stratify=labels
)

x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(
    x_test_and_val, y_test_and_val, z_test_and_val, test_size=0.5, stratify=y_test_and_val
)

assert x_train.shape[0] == y_train.shape[0] == len(z_train)

In [38]:
training_data = TensorDataset(torch.tensor(x_train), torch.tensor(z_train), torch.tensor(y_train))
training_sampler = RandomSampler(training_data)
training_dataloader = DataLoader(training_data, sampler=training_sampler, batch_size=VAR['batch_size'])

validation_data = TensorDataset(
    torch.tensor(x_val),
    torch.tensor(z_val),
    torch.tensor(y_val)
)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(
    validation_data, sampler=validation_sampler, batch_size=VAR['batch_size']
)

prediction_data = TensorDataset(torch.tensor(x_test), torch.tensor(z_test), torch.tensor(y_test))
prediction_dataloader = DataLoader(prediction_data, batch_size=VAR['batch_size'])

# Classification Task: Initiating Model

In [39]:
from transformers import BertConfig, BertModel
configuration = BertConfig()
model = BertModel(configuration)
configuration = model.config

In [40]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels=len(label_map.keys())
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
model = torch.nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, 

In [42]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [
        p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
    ], 'weight_decay': 0.01},
    {'params': [
        p for n, p in param_optimizer if any(nd in n for nd in no_decay)
    ], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, correct_bias=False)

C:\Users\JS\anaconda3\envs\NLPProjectVenv\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [43]:
def accuracy(predicted_labels, labels):
    predicted_labels = np.argmax(predicted_labels.to('cpu').numpy(), axis=1).flatten()
    labels = labels.to('cpu').numpy().flatten()
    
    return np.sum(predicted_labels == labels) / len(labels)

In [44]:
from tqdm import trange
epochs = 4
training_losses = []

In [48]:
for epoch in trange(epochs, desc="Epoch"):
    model.train()
    training_loss = 0
    training_steps = 0
    
    for step, batch in enumerate(training_dataloader):
        inputs = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device)
        
        # print("TRAIN INPUTS", inputs)
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_masks, labels=labels)
        # print("TRAIN OUTPUTS", outputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()
        training_steps += 1
        
        training_losses.append(loss.item())
        
    average_training_loss = training_loss/training_steps
    print(f'Epoch:{epoch+1}, Average Training Loss={average_training_loss}')
    
    model.eval()
    validation_accuracy = 0
    validation_steps = 0
    
    for batch in validation_dataloader:
        inputs = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device)
        
#         print("VAL INPUTS", inputs)
        with torch.no_grad():
            val_outputs = model(inputs, attention_mask=attention_masks, labels=labels)
            
        logits = val_outputs.logits
        print("LOGITS", logits)
        temp_validation_accuracy = accuracy(logits, labels)
        validation_accuracy += temp_validation_accuracy
        validation_steps += 1
        
    average_validation_accuracy = validation_accuracy/validation_steps
    print(f'Epoch:{epoch+1}, Average Validation Accuracy={average_validation_accuracy}')

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

TRAIN OUTPUTS SequenceClassifierOutput(loss=tensor(3.6286, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 1.8454,  2.2061,  0.7433,  1.0613,  1.5283,  2.0653, -0.1181,  1.8412,
          2.9870,  0.6270,  1.2781, -0.0305,  2.2433,  1.7404,  0.2673, -0.2461,
          1.6554,  1.8461,  1.9245,  2.2585,  2.5418,  1.1703,  2.2930,  1.3119,
          0.1057],
        [ 1.2418,  2.6880,  0.9460,  2.4484,  2.6387,  2.4543,  0.7652,  2.1591,
          2.6499,  0.9125,  1.1700,  0.4812,  2.0098,  1.7857,  0.3272,  0.3090,
          0.4598,  1.0582,  2.2957,  2.6767,  2.4618,  1.5082,  2.0762,  1.7423,
          0.6134],
        [ 1.7983,  2.2307,  1.1898,  1.5823,  2.5781,  2.2608,  0.5176,  2.1165,
          2.7798,  0.9782,  1.2781, -0.2517,  2.0461,  1.5560,  0.4805, -0.1195,
          0.9486,  1.3750,  2.3641,  2.6312,  2.4572,  1.5764,  2.2369,  1.5507,
         -0.0477],
        [ 2.0439,  2.7796,  0.6249,  1.2962,  1.8488,  2.5226,  0.5939,  2.2615,
          2.7694,  1.

Epoch:  25%|██▌       | 1/4 [01:30<04:30, 90.09s/it]

Epoch:1, Average Validation Accuracy=0.06451612903225806
TRAIN OUTPUTS SequenceClassifierOutput(loss=tensor(3.3228, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 1.4622,  0.6526,  3.7338,  0.4609,  0.7708,  1.0031,  2.7231,  2.2165,
          0.1917,  1.0277,  1.4663, -1.6596,  2.2434,  1.1784,  1.7633,  2.4215,
          2.9184,  2.8001,  0.5235,  1.0440,  0.9536,  1.2370,  1.1063,  1.6289,
          1.0880],
        [ 1.1168,  0.4575,  3.3379,  0.6884,  1.3221,  1.6813,  1.8410,  2.3661,
          0.7386,  1.4228,  1.2347, -1.6739,  2.6804,  1.7337,  1.3859,  2.5826,
          2.4778,  2.0398,  0.8556,  0.4033,  1.5717,  1.1913,  1.5061,  1.5513,
          1.5737],
        [ 0.8174,  0.9408,  3.4063,  0.2035,  1.3586,  2.0401,  2.8410,  1.8962,
          0.6569,  1.1948,  0.9967, -2.3780,  2.5792,  1.1411,  1.7975,  2.7512,
          1.7779,  2.3475,  0.6504,  0.7877,  0.8750,  1.1556,  1.3970,  1.8844,
          1.3274],
        [ 1.2611,  0.2158,  3.5033,  0.4387, 

Epoch:  50%|█████     | 2/4 [03:03<03:03, 91.96s/it]

Epoch:2, Average Validation Accuracy=0.03225806451612903
TRAIN OUTPUTS SequenceClassifierOutput(loss=tensor(3.5930, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 2.5382e+00, -9.7190e-02,  1.3465e+00,  2.1940e+00,  1.3989e+00,
          1.2843e+00,  2.6810e-02,  1.7602e+00,  1.4582e+00, -2.5856e-01,
          2.3361e+00,  2.3336e+00,  1.7071e+00,  2.1175e+00,  9.2229e-01,
          1.4614e+00,  2.3063e-01,  1.2324e+00,  2.8012e+00,  1.4679e+00,
          1.7493e+00,  2.6379e+00,  4.0192e-01,  1.2529e+00,  2.9788e+00],
        [ 2.3859e+00,  3.5206e-01,  8.9234e-01,  2.0308e+00,  1.6268e+00,
          1.5164e+00, -1.4497e-02,  1.5207e+00,  1.3783e+00, -3.2745e-01,
          2.1549e+00,  2.6495e+00,  1.9673e+00,  2.0031e+00,  9.5775e-01,
          1.3155e+00,  7.9037e-01,  1.4023e+00,  2.3866e+00,  2.2971e+00,
          2.2030e+00,  2.1375e+00, -1.4979e-01,  1.3146e+00,  2.7027e+00],
        [ 2.9753e+00,  3.7799e-01,  8.6153e-01,  2.0457e+00,  1.7137e+00,
          1.436

Epoch:  50%|█████     | 2/4 [03:21<03:21, 100.84s/it]


KeyboardInterrupt: 

In [ ]:
model.eval()
logits_list = []
labels_list = []

In [ ]:
for batch in prediction_dataloader:
    inputs = batch[0].to(device)
    attention_masks = batch[1].to(device)
    labels = batch[2].to(device)
    
    with torch.no_grad():
        outputs = model(inputs, attention_masks)
        logits = outputs.logits
        
    logits_list.append(logits.cpu().numpy())
    labels_list.append(labels.cpu().numpy())

In [ ]:
matthews_list = []
for i in range(len(labels_list)):
    mcc = matthews_corrcoef(labels_list[i], numpy.argmax(logits_list[i], axis=1).flatten())
    matthews_list.append(mcc)
    
for i, mcc in enumerate(matthews_list):
    print(f'batch {i+1}, MCC = {mcc}')
    
average_mcc = np.mean(matthews_list)
print('Average MCC =', average_mcc)

# OUT OF CODE

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
wn.synsets('prophetiqmadshiyn')

In [ ]:
sample_lemmas

In [ ]:
lemmatizer.lemmatize('deployed', 'v')

In [ ]:
sample_tag

In [ ]:
# Watch Cell
# print(clean_raw_text(sample_res))
# clean_raw_text(sample_res)
# nltk.corpus.words.raw().split('\n')
# np.isin(['detail'], nltk.corpus.words.raw().split('\n'))]
def x():
    x_dict = nltk.corpus.words.raw().split('\n')
    x_list = [levenshteinDistance('sklearn', word) for word in x_dict]
    
    id = x_list.index(min(x_list))
    print(id)
    print(min(x_list))
    print(nltk.corpus.words.raw().split('\n')[id])
    
x()

# lemmatizer.lemmatize('extracting')

In [ ]:
from nltk.metrics.distance import edit_distance as test
test('aws', 'reductionqunsobcudt', substitution_cost=1)